In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1153369,2021-04-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1153370,2021-04-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1153371,2021-04-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1153372,2021-04-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30967,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30969,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30971,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30973,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30975,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
711320,2021-04-05,Arkansas,Arkansas,5001,2081,36.00,5000,Arkansas,AR,Arkansas,State,3017804
711322,2021-04-06,Arkansas,Arkansas,5001,2081,36.00,5000,Arkansas,AR,Arkansas,State,3017804
711324,2021-04-07,Arkansas,Arkansas,5001,2082,36.00,5000,Arkansas,AR,Arkansas,State,3017804
711326,2021-04-08,Arkansas,Arkansas,5001,2082,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1153369,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1153370,2021-04-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1153371,2021-04-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1153372,2021-04-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
444,2021-04-09,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48
886,2021-04-09,Cook,Illinois,510006,10301.00,17031,IL,County,5150233,9902.58,200.01
1327,2021-04-09,Orange,California,267492,4826.00,6059,CA,County,3175692,8423.11,151.97
1767,2021-04-09,Maricopa,Arizona,527566,9716.00,4013,AZ,County,4485414,11761.81,216.61
2207,2021-04-09,Los Angeles,California,1225163,23431.00,6037,CA,County,10039107,12203.90,233.40
...,...,...,...,...,...,...,...,...,...,...,...
1152781,2021-04-09,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1152960,2021-04-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1153108,2021-04-09,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1153252,2021-04-09,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
444,2021-04-09,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48,68.48,4047.16
886,2021-04-09,Cook,Illinois,510006,10301.00,17031,IL,County,5150233,9902.58,200.01,200.01,9902.58
1327,2021-04-09,Orange,California,267492,4826.00,6059,CA,County,3175692,8423.11,151.97,151.97,8423.11
1767,2021-04-09,Maricopa,Arizona,527566,9716.00,4013,AZ,County,4485414,11761.81,216.61,216.61,11761.81
2207,2021-04-09,Los Angeles,California,1225163,23431.00,6037,CA,County,10039107,12203.90,233.40,233.40,12203.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152781,2021-04-09,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1152960,2021-04-09,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1153108,2021-04-09,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1153252,2021-04-09,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
743747,2021-04-09,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1
809467,2021-04-09,Clarke,Alabama,3461,60.00,1025,AL,County,23622,14651.60,254.00,254.00,14651.60,2
582081,2021-04-09,Lowndes,Alabama,1364,53.00,1085,AL,County,9726,14024.26,544.93,544.93,14024.26,3
433206,2021-04-09,Franklin,Alabama,4197,82.00,1059,AL,County,31362,13382.44,261.46,261.46,13382.44,4
516174,2021-04-09,Etowah,Alabama,13657,344.00,1055,AL,County,102268,13354.13,336.37,336.37,13354.13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094415,2021-04-09,Platte,Wyoming,611,11.00,56031,WY,County,8393,7279.88,131.06,131.06,7279.88,19
709819,2021-04-09,Converse,Wyoming,986,17.00,56009,WY,County,13822,7133.56,122.99,122.99,7133.56,20
882204,2021-04-09,Lincoln,Wyoming,1382,12.00,56023,WY,County,19830,6969.24,60.51,60.51,6969.24,21
981650,2021-04-09,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
582081,2021-04-09,Lowndes,Alabama,1364,53.00,1085,AL,County,9726,14024.26,544.93,544.93,14024.26,3,1
743747,2021-04-09,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1,2
247741,2021-04-09,Walker,Alabama,7083,275.00,1127,AL,County,63521,11150.64,432.93,432.93,11150.64,28,3
627678,2021-04-09,Greene,Alabama,908,34.00,1063,AL,County,8111,11194.67,419.18,419.18,11194.67,27,4
581321,2021-04-09,Crenshaw,Alabama,1492,57.00,1041,AL,County,13772,10833.58,413.88,413.88,10833.58,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626161,2021-04-09,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
882204,2021-04-09,Lincoln,Wyoming,1382,12.00,56023,WY,County,19830,6969.24,60.51,60.51,6969.24,21,20
860256,2021-04-09,Uinta,Wyoming,2142,12.00,56041,WY,County,20226,10590.33,59.33,59.33,10590.33,4,21
246580,2021-04-09,Teton,Wyoming,3660,9.00,56039,WY,County,23464,15598.36,38.36,38.36,15598.36,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,18,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,18,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,18,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,18,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,18,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150660,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1150661,2021-04-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1150662,2021-04-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1150663,2021-04-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
740663,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,1.00
740664,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
740665,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
740666,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
740667,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14927.31,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896928,2021-04-05,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5603.90,6,23,0.00
896929,2021-04-06,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5603.90,6,23,0.00
896930,2021-04-07,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5603.90,6,23,0.00
896931,2021-04-08,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5603.90,6,23,1.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
579749,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14024.26,1,3,1.00,0.00
579750,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14024.26,1,3,0.00,0.00
579751,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14024.26,1,3,0.00,0.00
579752,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14024.26,1,3,0.00,0.00
579753,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14024.26,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577844,2021-04-05,Albany,Wyoming,3990,11.00,56001,WY,County,38880,10262.35,28.29,28.29,10429.53,23,5,6.00,0.00
577845,2021-04-06,Albany,Wyoming,4006,11.00,56001,WY,County,38880,10303.50,28.29,28.29,10429.53,23,5,16.00,0.00
577846,2021-04-07,Albany,Wyoming,4021,11.00,56001,WY,County,38880,10342.08,28.29,28.29,10429.53,23,5,15.00,0.00
577847,2021-04-08,Albany,Wyoming,4038,11.00,56001,WY,County,38880,10385.80,28.29,28.29,10429.53,23,5,17.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
295697,2021-04-09,Imperial,California,27744,712.00,6025,CA,County,181215,15309.99,392.90,392.90,15309.99,1,2,40.00,0.00
2207,2021-04-09,Los Angeles,California,1225163,23431.00,6037,CA,County,10039107,12203.90,233.40,233.40,12203.90,2,5,713.00,43.00
147321,2021-04-09,San Bernardino,California,292358,4445.00,6071,CA,County,2180085,13410.39,203.89,203.89,13410.39,3,4,198.00,12.00
517650,2021-04-09,Inyo,California,1386,36.00,6027,CA,County,18039,7683.35,199.57,199.57,7683.35,4,25,0.00,0.00
65917,2021-04-09,Stanislaus,California,59759,1015.00,6099,CA,County,550660,10852.25,184.32,184.32,10852.25,5,9,95.00,3.00
30567,2021-04-09,Riverside,California,296090,4481.00,6065,CA,County,2470546,11984.80,181.38,181.38,11984.80,6,6,194.00,27.00
81714,2021-04-09,Tulare,California,49231,821.00,6107,CA,County,466195,10560.17,176.11,176.11,10560.17,7,10,29.00,1.00
52455,2021-04-09,San Joaquin,California,70746,1328.00,6077,CA,County,762148,9282.45,174.24,174.24,9282.45,8,19,137.00,3.00
29769,2021-04-09,Fresno,California,99935,1627.00,6019,CA,County,999101,10002.49,162.85,162.85,10002.49,9,14,70.00,0.00
469153,2021-04-09,Merced,California,31009,448.00,6047,CA,County,277680,11167.17,161.34,161.34,11167.17,10,8,24.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1093966,2021-04-09,Lassen,California,5660,1.00,6035,CA,County,30573,18513.07,3.27,3.27,18513.07,56,1,-1.00,0.00
295697,2021-04-09,Imperial,California,27744,712.00,6025,CA,County,181215,15309.99,392.90,392.90,15309.99,1,2,40.00,0.00
627620,2021-04-09,Kings,California,22683,243.00,6031,CA,County,152940,14831.31,158.89,158.89,14831.31,11,3,7.00,0.00
147321,2021-04-09,San Bernardino,California,292358,4445.00,6071,CA,County,2180085,13410.39,203.89,203.89,13410.39,3,4,198.00,12.00
2207,2021-04-09,Los Angeles,California,1225163,23431.00,6037,CA,County,10039107,12203.90,233.40,233.40,12203.90,2,5,713.00,43.00
30567,2021-04-09,Riverside,California,296090,4481.00,6065,CA,County,2470546,11984.80,181.38,181.38,11984.80,6,6,194.00,27.00
185193,2021-04-09,Kern,California,106984,1304.00,6029,CA,County,900202,11884.44,144.86,144.86,11884.44,14,7,260.00,6.00
469153,2021-04-09,Merced,California,31009,448.00,6047,CA,County,277680,11167.17,161.34,161.34,11167.17,10,8,24.00,1.00
65917,2021-04-09,Stanislaus,California,59759,1015.00,6099,CA,County,550660,10852.25,184.32,184.32,10852.25,5,9,95.00,3.00
81714,2021-04-09,Tulare,California,49231,821.00,6107,CA,County,466195,10560.17,176.11,176.11,10560.17,7,10,29.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
295697,2021-04-09,Imperial,California,27744,712.00,6025,CA,County,181215,15309.99,392.90,392.90,15309.99,1,2,40.00,0.00
2207,2021-04-09,Los Angeles,California,1225163,23431.00,6037,CA,County,10039107,12203.90,233.40,233.40,12203.90,2,5,713.00,43.00
147321,2021-04-09,San Bernardino,California,292358,4445.00,6071,CA,County,2180085,13410.39,203.89,203.89,13410.39,3,4,198.00,12.00
517650,2021-04-09,Inyo,California,1386,36.00,6027,CA,County,18039,7683.35,199.57,199.57,7683.35,4,25,0.00,0.00
65917,2021-04-09,Stanislaus,California,59759,1015.00,6099,CA,County,550660,10852.25,184.32,184.32,10852.25,5,9,95.00,3.00
30567,2021-04-09,Riverside,California,296090,4481.00,6065,CA,County,2470546,11984.80,181.38,181.38,11984.80,6,6,194.00,27.00
81714,2021-04-09,Tulare,California,49231,821.00,6107,CA,County,466195,10560.17,176.11,176.11,10560.17,7,10,29.00,1.00
52455,2021-04-09,San Joaquin,California,70746,1328.00,6077,CA,County,762148,9282.45,174.24,174.24,9282.45,8,19,137.00,3.00
29769,2021-04-09,Fresno,California,99935,1627.00,6019,CA,County,999101,10002.49,162.85,162.85,10002.49,9,14,70.00,0.00
469153,2021-04-09,Merced,California,31009,448.00,6047,CA,County,277680,11167.17,161.34,161.34,11167.17,10,8,24.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.90,15309.99,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3872,04/05/21,Lassen,5656,1.00,18499.98,3.27,3.27,18513.07,56,1,4.00,0.00
3873,04/06/21,Lassen,5660,1.00,18513.07,3.27,3.27,18513.07,56,1,4.00,0.00
3874,04/07/21,Lassen,5660,1.00,18513.07,3.27,3.27,18513.07,56,1,0.00,0.00
3875,04/08/21,Lassen,5661,1.00,18516.34,3.27,3.27,18513.07,56,1,1.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.90,15309.99,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.90,15309.99,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3958,04/05/21,Merced,30906,442.00,11130.08,159.18,161.34,11167.17,10,8,219.00,3.00
3959,04/06/21,Merced,30939,442.00,11141.96,159.18,161.34,11167.17,10,8,33.00,0.00
3960,04/07/21,Merced,30968,442.00,11152.41,159.18,161.34,11167.17,10,8,29.00,0.00
3961,04/08/21,Merced,30985,447.00,11158.53,160.98,161.34,11167.17,10,8,17.00,5.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)